In [ ]:
pip install ucimlrepo

In [9]:
from ucimlrepo import fetch_ucirepo
car_evaluation = fetch_ucirepo(id=19)

X = car_evaluation.data.features
Y = car_evaluation.data.targets
print(X)
print ("--------------")
print(Y)
print ("--------------")


     buying  maint  doors persons lug_boot safety
0     vhigh  vhigh      2       2    small    low
1     vhigh  vhigh      2       2    small    med
2     vhigh  vhigh      2       2    small   high
3     vhigh  vhigh      2       2      med    low
4     vhigh  vhigh      2       2      med    med
...     ...    ...    ...     ...      ...    ...
1723    low    low  5more    more      med    med
1724    low    low  5more    more      med   high
1725    low    low  5more    more      big    low
1726    low    low  5more    more      big    med
1727    low    low  5more    more      big   high

[1728 rows x 6 columns]
--------------
      class
0     unacc
1     unacc
2     unacc
3     unacc
4     unacc
...     ...
1723   good
1724  vgood
1725  unacc
1726   good
1727  vgood

[1728 rows x 1 columns]
--------------


In [8]:
variables_df = car_evaluation.variables
print (variables_df)

       name     role  ... units missing_values
0    buying  Feature  ...  None             no
1     maint  Feature  ...  None             no
2     doors  Feature  ...  None             no
3   persons  Feature  ...  None             no
4  lug_boot  Feature  ...  None             no
5    safety  Feature  ...  None             no
6     class   Target  ...  None             no

[7 rows x 7 columns]


In [10]:
import pandas as pd
combined_df= pd.concat([X, Y], axis = 1)
with pd.ExcelWriter('car_evaluation.xlsx', engine= 'openpyxl') as writer:
    combined_df.to_excel(writer, sheet_name="Variables Metadata", index=False)

In [34]:
from sklearn.model_selection import train_test_split #Splits your dataset into training and testing sets.
from sklearn.ensemble import RandomForestClassifier #"forest" of decision trees and combines their outputs for classification tasks.
from sklearn.metrics import classification_report, confusion_matrix #classification_report: Gives precision, recall, f1-score, and accuracy for each class. confusion_matrix: Shows the number of correct/incorrect predictions for each class in a matrix form.
from sklearn.preprocessing import StandardScaler #Imports a tool to standardize features by removing the mean and scaling to unit variance.

# scalar transformation or confusion matrix doesnt work on strings, will have to first convert it into numerical form

df= combined_df.copy()

# Label encode all categorical columns

#------
label_encoders = {}
for column in df.columns:
    if df[column].dtype == 'object':
           le = LabelEncoder()
           df[column] = le.fit_transform(df[column])
           label_encoders[column] = le # this stores the data in dictionary that can turn the numeric data into strings when needed
           

#----

x = df.drop(columns=["class"])
y = df["class"]

scalar = StandardScaler()
x_scaled = scalar.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size = 0.2, random_state= 42)
model  = RandomForestClassifier()
model.fit (x_train, y_train)

y_pred = model.predict(x_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))



[[ 75   7   1   0]
 [  0  11   0   0]
 [  0   0 235   0]
 [  1   0   0  16]]
              precision    recall  f1-score   support

           0       0.99      0.90      0.94        83
           1       0.61      1.00      0.76        11
           2       1.00      1.00      1.00       235
           3       1.00      0.94      0.97        17

    accuracy                           0.97       346
   macro avg       0.90      0.96      0.92       346
weighted avg       0.98      0.97      0.98       346



In [37]:
import pandas as pd

new_data = pd.DataFrame([{
    'buying': 'vhigh',
    'maint': 'vhigh',
    'doors': '2',
    'persons': '2',
    'lug_boot': 'small',
    'safety': 'low'
}])

# Apply label encoding using the same encoders
for column in new_data.columns:
    if column in label_encoders:
        le = label_encoders[column]
        if new_data[column].iloc[0] not in le.classes_:
            raise ValueError(f"'{new_data[column].iloc[0]}' is a new label not seen during training in column '{column}'")
        new_data[column] = le.transform(new_data[column])


# 3. Scale the input features using the same scaler
new_data_scaled = scalar.transform(new_data)

# 4. Predict the class
predicted_class_index = model.predict(new_data_scaled)[0]

# 5. (Optional) Convert label back to original string if needed
original_label = label_encoders['class'].inverse_transform([predicted_class_index])[0]
print("Predicted class:", original_label)



Predicted class: unacc
